In [1]:
from glob import glob
from tqdm import tqdm
import json
import re

In [2]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'Kode',
    'kode',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik',
    'jawaban',
    'teknis',
    'berkualitas',
    'mencoba',
    'kampanye',
    'komunitas',
    'stabilitas',
    'Stabilitas',
    'metode',
    'pria',
    'butuh',
    'jadwal',
    'kasus',
    'otomatis',
    'populer',
    'bisnis',
    'probabilitas',
    'rusak',
    'kapasitas',
    'rutinitas'
]
rejected_words = set(rejected_words)

def check_indon(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    return len(set(string.split()) & rejected_words) > 0

In [4]:
files = sorted(glob('output/*.json'), key = lambda x: int(x.split('/')[-1].replace('.json', '')))
len(files)

51748

In [5]:
rejected_instructions = [
    'tidak dapat melakukan penulisan semula', 
    'maaf', 
    'not able to fulfill your request',
    'saya tidak dapat membantu',
    'dengan permintaan itu',
    'feel free to ask',
    'saya tidak bisa',
]

In [6]:
count, indon_count = 0, 0

indon_texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)

    if data['output'] is None:
        continue
        
    if any([r in data['instruction'].lower() for r in rejected_instructions]):
        continue

    set_ins = set(data['instruction'].split())
    set_output = set(data['output'].split())

    indon_texts.append(data['instruction'])
    indon_texts.append(data['output'])

    count += 1

100%|██████████████████████████████████| 51748/51748 [00:01<00:00, 27034.02it/s]


In [7]:
indon_texts = set(indon_texts)
indon_texts = [t for t in indon_texts if len(t)]
len(indon_texts)

103227

In [61]:
with open('instructions-indon.texts', 'w') as fopen:
    for t in indon_texts:
        fopen.write(f'{json.dumps(t)}\n')

In [62]:
!cp instructions-indon.texts ~/ssd3/ctranslate2

In [10]:
mapping = {}
with open('/home/husein/ssd3/ctranslate2/instructions-indon.texts.requested') as fopen:
    for l in fopen:
        try:
            l = json.loads(l)
            mapping[l['src']] = l['r']
        except:
            pass
        
len(mapping)

103226

In [14]:
rejected = 0

with open('synthetic-alpaca_data_cleaned.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        with open(f) as fopen:
            data = json.load(fopen)
            
        if data['output'] is None:
            continue
        
        output_ms = mapping.get(data['output'])
        instruction_ms = mapping.get(data['instruction'])
        
        indon_ins = False
        if check_indon(data['instruction']):
            indon_ins = True
        
        indon_output = False
        if check_indon(data['output']):
            indon_output = True
        
        rejected_ins = False
        if 1024 <= len(enc.encode(data['instruction'])) <= 1025:
            rejected_ins = True
        
        rejected_output = False
        if 1024 <= len(enc.encode(data['output'])) <= 1025:
            rejected_output = True
            
        if rejected_ins or rejected_output:
            print(data)
            rejected += 1
            
        data['indon_ins'] = indon_ins
        data['indon_output'] = indon_output
        data['instruction_ms'] = instruction_ms
        data['output_ms'] = output_ms
        data['rejected_ins'] = rejected_ins
        data['rejected_output'] = rejected_output
        
        fopen_l.write(f'{json.dumps(data)}\n')
        count += 1

100%|███████████████████████████████████| 51748/51748 [00:11<00:00, 4630.27it/s]


In [15]:
!head -n 1 synthetic-alpaca_data_cleaned.jsonl

{"instruction": "Beri tiga petua untuk menjaga kesihatan yang sepadan di negara Malaysia.", "output": "Tiga petua untuk menjaga kesihatan yang sepadan di negara Malaysia adalah:\n1. Makan makanan yang seimbang dan berkhasiat: Berusaha untuk mengamalkan pemakanan yang seimbang dengan menggabungkan semua kumpulan makanan seperti bijirin, sayur-sayuran, buah-buahan, protein dan lemak yang sihat. Hindari makanan yang tinggi gula, garam, dan lemak tepu. Pastikan bahawa makanan yang diambil adalah segar dan bersih.\n2. Budayakan senaman secara berkala: Amalkan senaman secara teratur seperti berjalan kaki, berlari, berenang, atau mengikuti kelas senaman. Senaman dapat membantu mengurangkan risiko penyakit jantung, diabetes, obesiti, dan meningkatkan kesihatan mental dan fizikal. Pastikan anda mendapatkan cahaya matahari yang mencukupi untuk memperoleh vitamin D yang penting bagi kesihatan tulang.\n3. Amalkan gaya hidup sihat: Hindari merokok dan minum alkohol. Jarakkan diri dari aktiviti yang